In [ ]:
!pip install opencv-python==3.4.2.17 opencv-contrib-python==3.4.2.17

from google.colab import drive
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm
import csv


from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


     |████████████████████████████████| 25.0 MB 99 kB/s 
     |████████████████████████████████| 30.6 MB 27 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
drive.mount('/drive') 

Mounted at /drive


In [ ]:
train_Les = np.load('/drive/My Drive/CAD_1_1/train_Les_hairRemoved.npz',allow_pickle=True)
train_NV  = np.load('/drive/My Drive/CAD_1_1/train_NV_hairRemoved.npz',allow_pickle=True)
val_Les   = np.load('/drive/My Drive/CAD_1_1/val_Les_hairRemoved.npz',allow_pickle=True)
val_NV    = np.load('/drive/My Drive/CAD_1_1/val_NV_hairRemoved.npz',allow_pickle=True)

In [ ]:
trainX_Les  = np.array(train_Les['arr_0'])
trainX_NV   = np.array(train_NV['arr_0'])
valX_Les    = np.array(val_Les['arr_0'])
valX_NV     = np.array(val_NV['arr_0'])

In [ ]:
print(trainX_Les.shape)
print(trainX_NV.shape)
print(valX_Les.shape)
print(valX_NV.shape)

(2400, 450, 600, 3)
(2400, 450, 600, 3)
(600, 450, 600, 3)
(600, 450, 600, 3)


In [ ]:
sift=cv2.xfeatures2d.SIFT_create()

In [ ]:
siftFeatures_trainX_Les  = []
siftFeatures_trainX_NV   = []
siftFeatures_valX_Les    = []
siftFeatures_valX_NV     = []

for x in tqdm(trainX_Les):
  kp, des = sift.detectAndCompute(x, None)
  if des is not None:
    siftFeatures_trainX_Les.append(des)

for x in tqdm(trainX_NV):
  kp, des = sift.detectAndCompute(x, None)
  if des is not None:
    siftFeatures_trainX_NV.append(des)

for x in tqdm(valX_Les):
  kp, des = sift.detectAndCompute(x, None)
  if des is not None:
    siftFeatures_valX_Les.append(des)

for x in tqdm(valX_NV):
  kp, des = sift.detectAndCompute(x, None)
  if des is not None:
    siftFeatures_valX_NV.append(des)


100%|██████████| 600/600 [00:40<00:00, 14.88it/s]


In [ ]:
siftFeatures_trainX_Les  = np.vstack(siftFeatures_trainX_Les)
siftFeatures_trainX_NV   = np.vstack(siftFeatures_trainX_NV)
siftFeatures_valX_Les    = np.vstack(siftFeatures_valX_Les)
siftFeatures_valX_NV     = np.vstack(siftFeatures_valX_NV)

In [ ]:
print(siftFeatures_trainX_Les.shape)
print(siftFeatures_trainX_NV.shape)
print(siftFeatures_valX_Les.shape)
print(siftFeatures_valX_NV.shape)

(150868, 128)
(73478, 128)
(39553, 128)
(19581, 128)


In [ ]:
kmeans_siftFeatures_trainX = MiniBatchKMeans(n_clusters=2*10, batch_size=trainX_Les.shape[0]*3*2, verbose=0).fit(np.concatenate((siftFeatures_trainX_Les,siftFeatures_trainX_NV)))

In [ ]:

histo_siftFeatures_trainX = []


for x in tqdm(np.concatenate((trainX_Les,trainX_NV),axis=0)):
  kp, des = sift.detectAndCompute(x, None)

  histo = np.zeros(2*10)
  nkp = np.size(kp)

  if des is not None:
    kmeans_siftFeatures_trainX.predict(des)
    # hist = plt.hist()
    for d in des:
      idx = kmeans_siftFeatures_trainX.predict([d])
      histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

  histo_siftFeatures_trainX.append(histo)




100%|██████████| 4800/4800 [08:09<00:00,  9.81it/s]


In [ ]:
histo_siftFeatures_valX = []


for x in tqdm(np.concatenate((valX_Les,valX_NV),axis=0)):
  kp, des = sift.detectAndCompute(x, None)

  histo = np.zeros(2*10)
  nkp = np.size(kp)

  if des is not None:
    kmeans_siftFeatures_trainX.predict(des)
    # hist = plt.hist()
    for d in des:
      idx = kmeans_siftFeatures_trainX.predict([d])
      histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

  histo_siftFeatures_valX.append(histo)

100%|██████████| 1200/1200 [02:08<00:00,  9.37it/s]


In [ ]:
print(np.vstack(histo_siftFeatures_trainX).shape)
print(np.vstack(histo_siftFeatures_valX).shape)


(4800, 20)
(1200, 20)


In [ ]:
trainData = pd.DataFrame(histo_siftFeatures_trainX)
trainData['labels']=0
trainData['labels'][0:2400]=1

In [ ]:
valData = pd.DataFrame(histo_siftFeatures_valX)
valData['labels']=0
valData['labels'][0:600]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#trainData.to_csv('/drive/My Drive/CAD_1_1/rawSIFT_hairRemoved_Train.csv',index=False)
#valData.to_csv('/drive/My Drive/CAD_1_1/rawSIFT_hairRemoved_val.csv',index=False)

In [ ]:
trainDataShuffle=trainData.sample(frac=1)
valDataShuffle=valData.sample(frac=1)

In [ ]:
trainDataShuffle.head(15)

In [ ]:
mlp = MLPClassifier(verbose=False, max_iter=600000)
mlp.fit(trainDataShuffle.drop(["labels"],axis=1), trainDataShuffle['labels'])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=600000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
mlp.score(valDataShuffle.drop(["labels"],axis=1),valDataShuffle['labels'])

0.7341666666666666

In [ ]:
rndst=RandomForestClassifier(n_estimators=100)
rndst.fit(trainDataShuffle.drop(["labels"],axis=1), trainDataShuffle['labels'])
rndst.score(valDataShuffle.drop(["labels"],axis=1),valDataShuffle['labels'])

0.73

In [ ]:

best_svc = SVC()
best_svc.fit(trainDataShuffle.drop(["labels"],axis=1), trainDataShuffle['labels'])
best_svc.score(valDataShuffle.drop(["labels"],axis=1),valDataShuffle['labels'])

0.7316666666666667